## Install neccessary packages 

In [3]:
!pip install pandas numpy requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 22.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 19.9 MB/s eta 0:00:00a 0:00:01


### Fetch and process data

### Import neccessary packages

In [4]:
import pandas as pd
import numpy as np
import requests
from io import StringIO

### Make helper function for load data. -> return DataFrame

In [5]:
def load_data(base_url, year, station_id):
    url = f"{base_url}/{year}/{station_id}.csv"

    response = requests.get(url)
    data = StringIO(response.text)

    return pd.read_csv(data)

### Function for cleaning and transforming data (convert types/values, delete not valid records etc)

In [8]:
def clear_and_tranform_data(df):
    # Конвертація колонки DATE в формат datetime
    df["DATE"] = pd.to_datetime(df["DATE"])

    # Робимо конвертацію колонок TMP та DEW в строку, для того щоб видалити символи контролю якості (останні дві позиції)
    df["TMP"] = df["TMP"].astype(str)
    df["DEW"] = df["DEW"].astype(str)

    # Видаляємо символи контролю якості
    df["TMP"] = df["TMP"].str.slice(0, -2)
    df["DEW"] = df["DEW"].str.slice(0, -2)

    # Робимо конвертацію колонок TMP та DEW в числове значення
    df["TMP"] = pd.to_numeric(df["TMP"], errors="coerce")
    df["DEW"] = pd.to_numeric(df["DEW"], errors="coerce")

    # Позначимо значення для DEW -> 9999; TMP -> 9999, як пропущенні, оскільки в документації до даних вказано, що данні значення вважаються пропущеними
    df["TMP"] = df["TMP"].replace(9999, np.nan)
    df["DEW"] = df["DEW"].replace(9999, np.nan)
    
    # Видаляємо записи з пропущеними значеннями для колонок TMP та DEW
    df.dropna(subset=["TMP", "DEW"], inplace=True)

    # Видаляємо дублікати
    df.drop_duplicates(inplace=True)

    # Оскільки датасет поданий в масштабованому форматі (в документації для полів TMP та DEW вказаний SCALING_FACTOR: TMP, DEW -> 10). То проведемо трансформацію
    df["TMP"] = df["TMP"] / 10
    df["DEW"] = df["DEW"] / 10

    return df

### Function for calculate diff of TMP and DEW

In [9]:
def calculate_tmp_dew_diff(df):
    df["Temp_Dew_Diff"] = df["TMP"] - df["DEW"]
    return df

### Aggregate data by day of week and calculate mean ratio between TMP and DEW

In [10]:
def aggregate_data(df):
    # Додаємо нову колонку для ідентифікація дня тижня
    df["Day_Of_Week"] = df["DATE"].dt.day_name()

    # Агрегуємо дані по дням тижня
    aggregated_df = df.groupby("Day_Of_Week").agg({
        "TMP": ["mean", "max", "min"],
        "DEW": ["mean", "max", "min"]
    })

    # Переназиваємо колонки, оскільки після агрегації вони багато рівневими
    aggregated_df.columns = ['_'.join(col) for col in aggregated_df.columns]

    # Вказуємо порядок днів тижня, оскільки без цього воно виводить їх у алфавітному
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    aggregated_df = aggregated_df.reindex(day_order)

    # Знаходимо середнє співвідношення між колонками TMP та DEW
    aggregated_df["Temp_Dew_Ratio"] = aggregated_df["TMP_mean"] / aggregated_df["DEW_mean"]
    
    return aggregated_df

### Call helper functions (load data)

In [6]:
# Константи
BASE_URL = "https://www.ncei.noaa.gov/data/global-hourly/access"
YEAR = "2024"
STATION_ID = "01015099999"

# Визиваємо функції завантаження датасету
df = load_data(BASE_URL, YEAR, STATION_ID)

# Виводу коротку інформацію даних
print(df.head())
print(df.dtypes)

      STATION                 DATE  SOURCE  LATITUDE  LONGITUDE  ELEVATION  \
0  1015099999  2024-01-01T00:00:00       4      69.6  17.833333       14.0   
1  1015099999  2024-01-01T01:00:00       4      69.6  17.833333       14.0   
2  1015099999  2024-01-01T02:00:00       4      69.6  17.833333       14.0   
3  1015099999  2024-01-01T03:00:00       4      69.6  17.833333       14.0   
4  1015099999  2024-01-01T04:00:00       4      69.6  17.833333       14.0   

                NAME REPORT_TYPE  CALL_SIGN QUALITY_CONTROL  ...          CIG  \
0  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
1  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
2  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
3  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
4  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   

            VIS      TMP      DEW      SLP  

In [11]:
df_transformed = clear_and_tranform_data(df.copy())

# Після очистки даних видалився лише один запис
print(df.shape[0])
print(df_transformed.shape[0])

# Коротка інформація про очищені та трансформовані дані
print(df_transformed.head())
print(df_transformed.dtypes)

df_calcuted = calculate_tmp_dew_diff(df_transformed.copy())

# Коротка інформація про дані після розрахунку різниці між TMP та DEW
print(df_calcuted.head())
print(df_calcuted.dtypes)

df_aggregated = aggregate_data(df_calcuted.copy())

# Коротка інформація про дані після агрегації
print(df_aggregated)
print(df_aggregated.dtypes)

df_aggregated.to_csv("date_of_week_weather_date.csv")

7531
7530
      STATION                DATE  SOURCE  LATITUDE  LONGITUDE  ELEVATION  \
0  1015099999 2024-01-01 00:00:00       4      69.6  17.833333       14.0   
1  1015099999 2024-01-01 01:00:00       4      69.6  17.833333       14.0   
2  1015099999 2024-01-01 02:00:00       4      69.6  17.833333       14.0   
3  1015099999 2024-01-01 03:00:00       4      69.6  17.833333       14.0   
4  1015099999 2024-01-01 04:00:00       4      69.6  17.833333       14.0   

                NAME REPORT_TYPE  CALL_SIGN QUALITY_CONTROL  ...          CIG  \
0  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
1  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
2  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
3  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   
4  HEKKINGEN FYR, NO       FM-12      99999            V020  ...  99999,9,9,N   

            VIS  TMP   DEW      SLP     

In [13]:
print(df.shape[0])
count_99999 = (df["SLP"] == "99999,9").sum()
print(count_99999)

7531
7531
